In [1]:
import numpy as np

In [15]:
def lu_method(a_ : np.ndarray[np.ndarray], b_ : np.ndarray, show_annotation : bool = False) -> np.ndarray:
    a_ = np.array(a_, float)
    b_ = np.array(b_, float)
    
    if a_.shape[0] != a_.shape[1]:
        raise ValueError("Matrix a must be square")
    
    if a_.shape[0] != b_.shape[0]:
        raise ValueError("The dimensions of the two arrays must match")

    for k in range(1, a_.shape[0] + 1):
        minor_det = np.linalg.det(a_[:k, :k])
        if abs(minor_det) < 1e-13:
            raise ValueError(f"Leading principal minor #{k} is zero — LU decomposition without pivoting not possible.")
    
    n = len(b_)
    a = np.zeros((n, n + 1), dtype=float)
    a[:, :-1] = a_
    a[:, -1] = b_
    
    if show_annotation:
        print("System:\n", a_, b_, "\n")
        
    b = np.zeros((n, n), dtype=float)
    c = np.zeros((n, n + 1), dtype=float)
    
    b[:, 0] = a[:,0]
    c[0, :] = a[0, :] / b[0, 0]
    
    for i in range(1, n):
        for j in range(1, i + 1):
            b[i, j] = a[i, j] - np.sum([b[i, k] * c[k, j] for k in range(j)])
        
        c[i, i] = 1
        for j in range(i + 1, n + 1):
            c[i, j] = (a[i, j] - np.sum([b[i, k] * c[k, j] for k in range(j)])) / b[i, i]
            
    if show_annotation:
        print("B: ", b)
        print("\nC: ", c)
        
    y = np.zeros(n, dtype=float)
    y[0] = b_[0] / b[0, 0]
    
    for i in range(1, n):
        y[i] = (b_[i] - np.sum(b[i, :i] * y[:i])) / b[i, i]
        
    x = np.zeros(n, dtype=float)
    x[n - 1] = y[n - 1]
    for i in range(n - 2, -1, -1):
        x[i] = y[i] - np.sum(c[i, i+1:-1] * x[i + 1:])
        
    if show_annotation:
        print("x: ", x)
    
    return x

In [17]:
a = np.array([[3,1,-1,2], [-5,1,3,-4], [2,0,1,-1], [1,-5,3,-3]])
b = np.array([6,-12,1,3])

x = lu_method(a, b, show_annotation=True)

b_ = a @ x
max_error = np.max(np.abs(b - b_))
print("max error: ", max_error)

System:
 [[ 3.  1. -1.  2.]
 [-5.  1.  3. -4.]
 [ 2.  0.  1. -1.]
 [ 1. -5.  3. -3.]] [  6. -12.   1.   3.] 

B:  [[ 3.          0.          0.          0.        ]
 [-5.          2.66666667  0.          0.        ]
 [ 2.         -0.66666667  2.          0.        ]
 [ 1.         -5.33333333  6.          2.5       ]]

C:  [[ 1.          0.33333333 -0.33333333  0.66666667  2.        ]
 [ 0.          1.          0.5        -0.25       -0.75      ]
 [ 0.          0.          1.         -1.25       -1.75      ]
 [ 0.          0.          0.          1.          3.        ]]
x:  [ 1. -1.  2.  3.]
max error:  8.881784197001252e-16


In [19]:
def return_max_error(a, b):
    x = lu_method(a, b)
    b_ = a @ x
    max_error = np.max(np.abs(b - b_))
    return max_error


a2 = np.array([[4, 2],
               [2, 3]])
b2 = np.array([6, 8])

a3 = np.array([[6, 2, 1],
               [2, 5, 2],
               [1, 2, 4]])
b3 = np.array([9, 10, 7])

a4 = np.array([[10, 2, 3, 1],
               [2, 8, 1, 0],
               [3, 1, 9, 4],
               [1, 0, 4, 7]])
b4 = np.array([12, 9, 11, 8])

a5 = np.array([[9, 1, 2, 3, 1],
               [1, 7, 1, 2, 0],
               [2, 1, 8, 1, 2],
               [3, 2, 1, 10, 3],
               [1, 0, 2, 3, 6]])
b5 = np.array([17, 13, 15, 18, 10])

a6 = np.array([[12, 3, 2, 1, 0, 2],
               [3, 9, 1, 0, 2, 1],
               [2, 1, 11, 4, 1, 2],
               [1, 0, 4, 10, 2, 3],
               [0, 2, 1, 2, 8, 1],
               [2, 1, 2, 3, 1, 9]])
b6 = np.array([15, 12, 18, 14, 10, 16])

systems = [
    (a2, b2),
    (a3, b3),
    (a4, b4),
    (a5, b5),
    (a6, b6)
]

for n in [8, 12, 16, 20, 40, 60, 80, 100, 200, 400, 600, 800]:
    A = np.random.randn(n, n)
    x_true = np.ones(n)
    b = A @ x_true
    systems.append((A, b))

for a, b in systems:
    err = return_max_error(a, b)
    print(f"{a.shape[0]}x{a.shape[0]} max error = {err:.3e}")

2x2 max error = 0.000e+00
3x3 max error = 0.000e+00
4x4 max error = 1.776e-15
5x5 max error = 3.553e-15
6x6 max error = 3.553e-15
8x8 max error = 3.553e-15
12x12 max error = 4.263e-14
16x16 max error = 8.060e-14
20x20 max error = 8.837e-14
40x40 max error = 3.610e-12
60x60 max error = 1.950e-12
80x80 max error = 2.033e-12
100x100 max error = 1.106e-11
200x200 max error = 1.581e-11
400x400 max error = 5.319e-11
600x600 max error = 9.258e-09
800x800 max error = 2.163e-08
